In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/집교1팀플

/content/drive/MyDrive/집교1팀플


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchsummary import summary
import numpy as np
import random
import time
import pickle
from torch.nn.utils.rnn import pad_sequence
import os
import pandas as pd

random.seed(10)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

In [ ]:
config = {
          'data_stats'  : [[0.05929394720093303], [0.2313007198154599]],
          'data_stats_64_3000' : [[0.059319619673050376], [0.23133441623990797]],
          'data_stats_128_1000' : [[0.029937755134580656], [0.1669103758131807]],
          'data_stats_328classes' : [[0.05900596462342797], [0.23563586476232082]],
          'batch_size'  : 256,
          'worker'      : 2,
          'epochs'      : 100,
          'momentum'    : 0.9,
          'lr_decay'    : 0.0005,
          'SGD_lr'      : 0.01,
          'Adam_lr'     : 0.001,
          'overfitting_count' : 10,
          }

In [ ]:
image_path = '/content/drive/MyDrive/집교1팀플/데이터및라벨/328classes_matrix_64_10000.npy'
image = torch.from_numpy(np.load(image_path))

In [ ]:
label_path = '/content/drive/MyDrive/집교1팀플/데이터및라벨/328classes_label_64_10000.npy'
label = torch.from_numpy(np.load(label_path))

In [ ]:
print(image.shape)
print(label.shape)

torch.Size([3280000, 1, 64, 64])
torch.Size([3280000])


In [ ]:
data_set = TensorDataset(image, label)

In [ ]:
# 데이터셋 크기 분할 (70% 훈련, 20% 검증, 10% 테스트)
train_size = int(0.7 * len(data_set))
val_size = int(0.2 * len(data_set))
test_size = len(data_set) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(data_set, [train_size, val_size, test_size])

train_transforms = transforms.Compose([
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(*config['data_stats_328classes']),
])

valid_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(*config['data_stats_328classes']),
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(*config['data_stats_328classes']),
])

train_dataset.dataset.transform = train_transforms
val_dataset.dataset.transform = valid_transforms
test_dataset.dataset.transform = test_transforms

# DataLoader 생성
trainloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
valloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)
testloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

In [ ]:
import torch.nn as nn
from torchvision.models import ResNet

class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class SEBottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, groups=1,
                 base_width=64, dilation=1, norm_layer=None,
                 *, reduction=16):
        super(SEBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se = SELayer(planes * 4, reduction)

        self.stride = stride

        self.shortcut = nn.Sequential()
        if stride != 1 or inplanes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inplanes, planes * self.expansion, stride = stride, kernel_size = 1, bias = False),
                nn.BatchNorm2d(planes * self.expansion)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
        out = self.se(out)

        out += self.shortcut(x)
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=328):
        super().__init__()

        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size = 7, stride = 2, padding = 3, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, 328)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
def se_resnet152(num_classes=328):
    model = ResNet(SEBottleneck, [3, 8, 36, 3], num_classes=num_classes)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    return model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = se_resnet152(num_classes=328)
model.to(device)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (conv2_x): Sequential(
    (0): SEBottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
       

In [ ]:
def save_checkpoint(model, optimizer, epoch, best_loss, model_name, idx):
    checkpoint_path = os.path.join(checkpoint_dir, f"{model_name}_checkpoint{idx}.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_loss': best_loss
    }, checkpoint_path)


def load_checkpoint(model, optimizer, model_name, idx):
    checkpoint_path = os.path.join(checkpoint_dir, f"{model_name}_checkpoint{idx}.pth")
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        best_loss = checkpoint['best_loss']
        return model, optimizer, epoch, best_loss
    else:
        print(f"No checkpoint found for {model_name}.")
        return model, optimizer, 0, np.inf

In [ ]:
from time import sleep
from tqdm.auto import tqdm

idx = 0
checkpoint_dir = "/content/drive/MyDrive/checkpointsforgol2"
model_name = "senet"

# from tqdm.notebook import tqdm  # tqdm의 노트북 전용 모듈

# 결합된 모델 학습
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=config['SGD_lr'], momentum=config['momentum'], weight_decay=config['lr_decay'])

overfitting_count = 0
best_loss = float('inf')

for epoch in range(config['epochs']):
    model.train()

    with tqdm(trainloader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}")
        for stroke_tensors, labels in tepoch:
            stroke_tensors, labels = stroke_tensors.float().to(device), labels.to(device)

            # 파라미터 경사 초기화
            optimizer.zero_grad()

            # CNN과 RNN 결합 모델의 forward 진행
            outputs = model(stroke_tensors)

            # 손실 계산
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            tepoch.set_postfix(loss=loss.item())

    # 검증
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for stroke_tensors, labels in valloader:
            stroke_tensors, labels = stroke_tensors.float().to(device), labels.to(device)


            outputs = model(stroke_tensors)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    torch.cuda.empty_cache()

    save_checkpoint(model, optimizer, epoch, best_loss, model_name, idx)
    idx += 1

    val_loss /= len(valloader)
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss:.4f}')

    if val_loss < best_loss:
        best_loss = val_loss
        overfitting_count = 0
        torch.save(model.state_dict(), f'./SENet152/senet_{epoch+1}.pth')
    else:
        overfitting_count += 1
        if overfitting_count >= config['overfitting_count']:
            print(f'Early stopping at epoch {epoch+1}')
            break

print('Finished Training')

# 테스트 정확도 계산
correct = 0
total = 0
with torch.no_grad():
    for stroke_tensors, labels in testloader:
        stroke_tensors, labels = stroke_tensors.float().to(device), labels.to(device)
        outputs = model(stroke_tensors)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test set: {100 * correct / total:.2f}%')



  0%|          | 0/8969 [00:00<?, ?batch/s]

Epoch 1, Validation Loss: 0.9112


  0%|          | 0/8969 [00:00<?, ?batch/s]

Epoch 2, Validation Loss: 0.8025


  0%|          | 0/8969 [00:00<?, ?batch/s]

...

Epoch 39, Validation Loss: 0.6894


  0%|          | 0/8969 [00:00<?, ?batch/s]

Epoch 40, Validation Loss: 0.6849


  0%|          | 0/8969 [00:00<?, ?batch/s]

Epoch 41, Validation Loss: 0.6750
Early stopping at epoch 41
Finished Training
Accuracy of the model on the test set: 80.75%


In [ ]:
# top-1 acc and top-5 acc
model.eval()
correct = 0
total = 0
top5_correct = 0
with torch.no_grad():
    for stroke_tensors, labels in testloader:
        stroke_tensors, labels = stroke_tensors.float().to(device), labels.to(device)
        outputs = model(stroke_tensors)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        _, top5_predicted = torch.topk(outputs.data, 5, dim=1)
        top5_correct += (top5_predicted == labels.view(-1, 1).expand_as(top5_predicted)).any(dim=1).sum().item()

print(f'Accuracy of the model on the test set: {100 * correct / total:.2f}%')
print(f'Top-5 Accuracy of the model on the test set: {100 * top5_correct / total:.2f}%')